In [ ]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium
pip install bs4


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [49.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,110 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,236 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/u

In [ ]:
# 느낌표를 붙이면 Shell 명령어로 실행됩니다.
!python --version

# Selenium을 확인하려면 다음과 같이 사용해야 합니다.
import selenium
print(selenium.__version__)

Python 3.10.12
4.11.2


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

In [ ]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('--headless') # 헤드리스 모드 활성화
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)


In [ ]:
# DATA_PATH = "/content/drive/Sharedwithme/4조_화사하조/04.기타자료/data/"

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
wait = WebDriverWait(driver, 10) # 10초 대기 설정

# URL 설정
url = 'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=1000001000100080001&fltDispCatNo=&prdSort=01&pageIdx=1&rowsPerPage=24&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0&trackingCd=Cat1000001000100080001_Small&amplitudePageGubun=SMALL_CATE&t_page=%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC%EA%B4%80&t_click=%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC%EC%83%81%EC%84%B8_%EC%86%8C%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC&midCategory=%ED%86%A0%EB%84%88%2F%EB%A1%9C%EC%85%98%2F%EC%98%AC%EC%9D%B8%EC%9B%90&smallCategory=%EC%86%8C_%EC%8A%A4%ED%82%A8%2F%ED%86%A0%EB%84%88&checkBrnds=&lastChkBrnd=&t_3rd_category_type=%EC%86%8C_%EC%8A%A4%ED%82%A8%2F%ED%86%A0%EB%84%88'

# 제품 링크 수집을 위한 페이지 열기
driver.get(url)

driver.maximize_window()

result_df = pd.DataFrame(columns=['상품명', '리뷰어 이름', '별점',  '리뷰', '피부타입', '피부톤', '피부트러블'])
processed_reviews = set()

def extract_reviews_from_page(product_name):
    global result_df
    reviews = driver.find_elements(By.CSS_SELECTOR, 'ul.inner_list > li')

    for review in reviews:
        # 리뷰어 이름
        try:
            reviewer_name_element = review.find_element(By.CSS_SELECTOR, '.info_user .id')
            reviewer_name = reviewer_name_element.text
        except:
            reviewer_name = "이름불명"

        # 별점
        try:
            star_rating_text = review.find_element(By.CSS_SELECTOR, '.score_area .point').text
        except:
            star_rating_text = "없음"

        # 피부타입 및 피부고민
        skin_type_elements = review.find_elements(By.CSS_SELECTOR, '.tag span')
        skin_type_info = skin_type_elements[0].text if len(skin_type_elements) > 0 else None
        skin_tone_info = skin_type_elements[1].text if len(skin_type_elements) > 1 else None
        skin_trouble_info = skin_type_elements[2].text + "+" + skin_type_elements[3].text if len(skin_type_elements) > 3 else None

        # 리뷰 텍스트
        try:
            review_text_element = review.find_element(By.CSS_SELECTOR, '.txt_inner')
            review_text = review_text_element.text
        except:
            review_text = '리뷰없음'

        review_identifier = (product_name, reviewer_name, review_text)

        processed_reviews.add(review_identifier)

        # 새로운 인덱스 (현재 DataFrame 길이)
        new_index = len(result_df)

        # 새로운 행 추가
        result_df.loc[new_index] = [product_name, reviewer_name, star_rating_text, review_text, skin_type_info, skin_tone_info, skin_trouble_info]

    duplicated_rows = result_df.duplicated(subset=['상품명', '리뷰어 이름', '리뷰'])
    result_df = result_df[~duplicated_rows].reset_index(drop=True)


def go_to_page(page_num):
    if page_num > 1:
        next_page_element = driver.find_element(By.CSS_SELECTOR, f'a[data-page-no="{page_num}"]')
        driver.execute_script("arguments[0].click();", next_page_element) # JavaScript로 클릭 실행
        # 페이지 로딩 대기: 페이지 번호가 바뀌었는지 확인
        wait.until(EC.text_to_be_present_in_element((By.CSS_SELECTOR, 'strong[title="현재 페이지"]'), str(page_num)))

        # 충분한 로딩 시간 대기
        time.sleep(2) # 여기에 추가적인 대기 시간을 둠

current_page = 1
processed_product_index = -1
processed_page = 0

while current_page <= 5:
    print(f"{current_page}페이지")
    go_to_page(current_page)
    product_elements = driver.find_elements(By.CSS_SELECTOR, 'div.prd_name') # 매번 새로 쿼리

    for i in range(len(product_elements)):
        if i <= processed_product_index and current_page == processed_page:
            continue  # 이미 처리된 상품은 건너뜀

        # 상품 목록 다시 찾기 (새로고침 된 페이지에서)
        product_elements = driver.find_elements(By.CSS_SELECTOR, 'div.prd_name')
        product_element = product_elements[i]

        product_name = product_element.find_element(By.CSS_SELECTOR, 'a .tx_name').text
        print(product_name)

        # 상세 페이지 링크 찾기
        detail_page_link = product_element.find_element(By.CSS_SELECTOR, 'a.goodsList')

        # 상세 페이지로 이동 (클릭)
        product_element = product_elements[i]
        detail_page_link = product_element.find_element(By.CSS_SELECTOR, 'a.goodsList')
        detail_page_link.click()
        time.sleep(2) # 페이지 로딩 대기

        # 리뷰 탭을 찾고 클릭하기
        review_tab = driver.find_element(By.CSS_SELECTOR, 'li#reviewInfo a.goods_reputation')
        review_tab.click()
        time.sleep(2) # 리뷰 탭 로딩 대기

        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(2) # 스크롤 내린 후 로딩 대기


        for review_page_num in range(1, 11):
            print(f"리뷰 페이지 {review_page_num} 수집 중...")

            extract_reviews_from_page(product_name)

            try:
                next_review_page_element = driver.find_element(By.CSS_SELECTOR, f'a[data-page-no="{review_page_num+1}"]')
                driver.execute_script("arguments[0].click();", next_review_page_element) # JavaScript로 클릭 실행
                time.sleep(2) # 페이지 로딩 대기
            except:
                print("마지막 리뷰 페이지입니다.")
                break

        # 원래 페이지로 돌아가기
        driver.get(url) # 이전 URL을 다시 로드
        time.sleep(2) # 페이지 로딩 대기 (대기 시간을 늘릴 수 있음)
        go_to_page(current_page) # 현재 페이지 번호로 다시 이동

        # 다음 상품 클릭을 위해 스크롤을 다시 내림
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    processed_product_index = -1
    processed_page = current_page
    current_page += 1

driver.quit()

1페이지
쏘내추럴 올 데이 메이크업 픽서 더블기획 (120ml+120ml)
리뷰 페이지 1 수집 중...
리뷰 페이지 2 수집 중...
리뷰 페이지 3 수집 중...
리뷰 페이지 4 수집 중...
리뷰 페이지 5 수집 중...
리뷰 페이지 6 수집 중...
리뷰 페이지 7 수집 중...
리뷰 페이지 8 수집 중...
리뷰 페이지 9 수집 중...
리뷰 페이지 10 수집 중...
[에이로그 PICK / 화해1위] 코스알엑스 더 레티놀 0.5 오일 20ml
리뷰 페이지 1 수집 중...
리뷰 페이지 2 수집 중...
리뷰 페이지 3 수집 중...
리뷰 페이지 4 수집 중...
리뷰 페이지 5 수집 중...
리뷰 페이지 6 수집 중...
리뷰 페이지 7 수집 중...
리뷰 페이지 8 수집 중...
리뷰 페이지 9 수집 중...
리뷰 페이지 10 수집 중...
아벤느 오 떼르말 300ml*2 + 50ml 기획
리뷰 페이지 1 수집 중...
리뷰 페이지 2 수집 중...
리뷰 페이지 3 수집 중...
리뷰 페이지 4 수집 중...
리뷰 페이지 5 수집 중...
리뷰 페이지 6 수집 중...
리뷰 페이지 7 수집 중...
리뷰 페이지 8 수집 중...
리뷰 페이지 9 수집 중...
리뷰 페이지 10 수집 중...
쏘내추럴 올 데이 메이크업 픽서 120ml (안개분사 캔타입)
리뷰 페이지 1 수집 중...
리뷰 페이지 2 수집 중...
리뷰 페이지 3 수집 중...
리뷰 페이지 4 수집 중...
리뷰 페이지 5 수집 중...
리뷰 페이지 6 수집 중...
리뷰 페이지 7 수집 중...
리뷰 페이지 8 수집 중...
리뷰 페이지 9 수집 중...
리뷰 페이지 10 수집 중...
차앤박(CNP) 프로폴리스 에너지 앰플 미스트 250ml
리뷰 페이지 1 수집 중...
리뷰 페이지 2 수집 중...
리뷰 페이지 3 수집 중...
리뷰 페이지 4 수집 중...
리뷰 페이지 5 수집 중...
이미 처리된 리뷰입니다. 건너뜁니다.
리뷰 페이지 6 수집 중...
리뷰 페이지 7 수집 중

ProtocolError: ignored

In [ ]:
result_df['상품분류'] = '토너/로션/올인원' # 지금 이 분류 url 에서 크롤링해서 이렇게 지정. 다른 분류 url 이면 바꿔야함.
result_df['별점'] = result_df['별점'].str.replace("평점", "")
result_df['별점'] = result_df['별점'].str.replace("5점만점에", "")
result_df['별점'] = result_df['별점'].str.replace("점", "")
result_df['리뷰'] = result_df['리뷰'].str.replace("\n", "")
result_df['리뷰'] = result_df['리뷰'].str.replace("\b", "")
result_df['리뷰'] = result_df['리뷰'].str.replace("\t", "")
result_df['리뷰'] = result_df['리뷰'].str.replace("\a", "")
result_df['피부타입'] = result_df['피부타입'].apply(lambda x: x.split(' / ')[0] if isinstance(x, str) else x)
result_df
# 열의 순서를 변경
cols = ['상품분류'] + [col for col in result_df.columns if col != '상품분류']
result_df = result_df[cols]

In [ ]:
 result_df.to_csv("파일경로")